Generator & Discriminator modules taken from: https://www.kaggle.com/code/joonasyoon/wgan-cp-with-celeba-and-lsun-dataset/notebook

Gradient Penalty method taken from: https://github.com/aadhithya/gan-zoo-pytorch/blob/master/models/wgan_gp.py

In [1]:
from pathlib import Path
USE_COLAB: bool = True
dataset_base_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/RealFaces")
if USE_COLAB:
  from google.colab import drive
  
  # Mount the drive to access google shared docs
  drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch import autograd
from torchvision.utils import save_image
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
from zipfile import ZipFile
from PIL import Image

In [3]:
output_dir = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/WGANGP/models")
output_dir.mkdir(exist_ok=True, parents=True)
zip_file = dataset_base_path / "img_align_celeb.zip"
gen_imgs_path = output_dir / "images"
gen_imgs_path.mkdir(exist_ok=True, parents=True)

config = {
    "n_workers": 2,
    "batch_size": 128,
    "img_size": 64,
    "n_channels": 3,
    "latent_dim": 100,
    "ngf": 64,  # size of feature maps in generator
    "ndf": 64,  # size of feature maps in discriminator
    "epochs": 50,
    "n_critics": 3,  # number of steps to train discriminator
    "lr": 2e-4,
    "beta1": 0.5,
    "n_img_samples": 25,
    "w_gp": 10,  # default value used in 'https://arxiv.org/pdf/1704.00028.pdf'
    "chkp_freq": 1,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fixed_noise = torch.randn(config["n_img_samples"], config["latent_dim"], 1, 1).to(device)

In [4]:
class CelebrityData(torch.utils.data.Dataset):

  def __init__(self,zip_file,transform = None) :
    self.zip_file = zip_file
    self.transform = transform

    # Create a ZipFile Object and load sample.zip in it
    with ZipFile(str(zip_file), 'r') as zipObj:
      # Extract all the contents of zip file in current directory
      zipObj.extractall()


    self.ListOfImages = zipObj.namelist()[1:]


  def __getitem__(self,index):

    image = Image.open(self.ListOfImages[index]).convert('RGB')

    if self.transform is not None:
      image = self.transform(image)

    return image


  def __len__(self):
    return  len(self.ListOfImages)

In [5]:
celebrity_data = CelebrityData(zip_file = zip_file,
                              transform = transforms.Compose([
                                transforms.Resize(int(config['img_size'] * 1.1)),
                                transforms.CenterCrop(config['img_size']),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = (0.5,0.5,0.5),
                                                     std  = (0.5,0.5,0.5))       
                                
]))

dataloader = torch.utils.data.DataLoader(dataset = celebrity_data,
                                         shuffle = True,
                                         batch_size = config['batch_size'],
                                         num_workers = config["n_workers"],
                                         drop_last = True,
                                         pin_memory = True) 

In [6]:
# custom weights initialization called on netG and netD
def weights_init(m):
  if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
    nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif isinstance(m, nn.BatchNorm2d):
    nn.init.normal_(m.weight.data, 0.0, 0.02)
    nn.init.constant_(m.bias.data, 0)

In [7]:
class Generator(nn.Module):
  def __init__(self, latent_dim: int = 100, ngf: int = 64, n_channels: int = 3):
    super(Generator, self).__init__()
    self.main = nn.Sequential(
        nn.ConvTranspose2d(latent_dim, ngf * 8, kernel_size=(4, 4), stride=(1, 1), padding=(0, 0)),
        nn.BatchNorm2d(ngf * 8),
        nn.LeakyReLU(0.2),

        nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        nn.BatchNorm2d(ngf * 4),
        nn.LeakyReLU(0.2),

        nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        nn.BatchNorm2d(ngf * 2),
        nn.LeakyReLU(0.2),

        nn.ConvTranspose2d(ngf * 2, ngf, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        nn.BatchNorm2d(ngf),
        nn.LeakyReLU(0.2),

        nn.ConvTranspose2d(ngf, n_channels, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        nn.Tanh(),
    )

  def forward(self, input):
    return self.main(input)

  
class Discriminator(nn.Module):
  def __init__(self, n_channels: int = 3, ndf: int = 64):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential(
        nn.Conv2d(n_channels, ndf, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        # No batch norm when using gradient penalty, b/c we want to calculate gradients of outputs w.r.t. original inputs
        nn.LeakyReLU(0.2),

        nn.Conv2d(ndf, ndf * 2, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        nn.LeakyReLU(0.2),

        nn.Conv2d(ndf * 2, ndf * 4, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        nn.LeakyReLU(0.2),

        nn.Conv2d(ndf * 4, ndf * 8, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        nn.LeakyReLU(0.2),

        nn.Conv2d(ndf * 8, 1, kernel_size=(4, 4), stride=(2, 2), padding=(0, 0)),
    )

  def forward(self, input):
    return self.main(input)

In [ ]:
netG = Generator(config["latent_dim"], config["ngf"], config["n_channels"]).to(device)
netG.apply(weights_init)

netD = Discriminator(config["n_channels"], config["ndf"]).to(device)
netD.apply(weights_init)

optimizerG = Adam(netG.parameters(), lr=config["lr"], betas=(config["beta1"], 0.999))
optimizerD = Adam(netD.parameters(), lr=config["lr"], betas=(config["beta1"], 0.999))

G_losses = []
critic_losses = []
critic_gps = []

epoch_tqdm = tqdm(total=config["epochs"], position=0, initial=1)
print("Starting Training Loop...")
for epoch in range(config["epochs"]):
  model_output_path = output_dir / f"WGANGP--{epoch+1}.pth"
  epoch_gen_imgs_path = gen_imgs_path / f"{epoch+1}"
  epoch_gen_imgs_path.mkdir(exist_ok=True, parents=True)

  for batch_idx, imgs in enumerate(dataloader):
    imgs = imgs.to(device)

    z = torch.randn(config["batch_size"], config["latent_dim"], 1, 1).to(device)
    gen_out = netG(z).detach()
    for critic_step in range(config["n_critics"]):
      # Train with all-real batch
      optimizerD.zero_grad()

      real_D_out = netD(imgs)
      fake_D_out = netD(gen_out)

      # Calculate gradient penalty
      eps = torch.rand(config["batch_size"], 1, 1, 1).to(device)
      eps = eps.expand_as(imgs)
      interpolation = eps * imgs + (1 - eps) * gen_out
      interpolation.requires_grad = True

      interp_logits = netD(interpolation)
      grad_outputs = torch.ones_like(interp_logits)

      gradients = autograd.grad(
          outputs=interp_logits,
          inputs=interpolation,
          grad_outputs=grad_outputs,
          create_graph=True,
          retain_graph=True,
      )[0]

      gradients = gradients.view(config["batch_size"], -1)
      grad_norm = gradients.norm(2, 1)

      gradient_penalty = config["w_gp"] * torch.mean((grad_norm - 1) ** 2)

      # calculate loss with gradient penalty
      loss_c = fake_D_out.mean() - real_D_out.mean()
      loss = loss_c + gradient_penalty

      loss.backward()
      optimizerD.step()

      critic_losses.append(loss.item())
      critic_gps.append(gradient_penalty.item())
    
    # train generator
    optimizerG.zero_grad()

    G_z = netG(z)
    D_G_z = netD(G_z)
    loss = -D_G_z.mean().view(-1)

    loss.backward()
    optimizerG.step()

    G_losses.append(loss.item())

    if (batch_idx + 1) % 50 == 0:
      print(" [%d/%d] [%d/%d]\tLoss_Critic: %.8f\tLoss_G: %.8f\tMean_GP: %.6f" % (
          epoch+1, config["epochs"], batch_idx + 1, len(dataloader), sum(critic_losses[-50:]) / 50, sum(G_losses[-50:]) / 50, sum(critic_gps[-50:]) / 50
      ))

  epoch_tqdm.update(1)
  if (epoch + 1) % config["chkp_freq"] == 0:
    torch.save(
        {
            "Generator": netG.state_dict(),
            "Discriminator": netD.state_dict(),
        },
        str(model_output_path),
    )

    netG.eval()
    generated_imgs = netG(fixed_noise)
    for idx, gen_img in enumerate(generated_imgs):
      img_output_path = epoch_gen_imgs_path / f"{idx}.png"
      save_image(gen_img, img_output_path)
    netG.train()


  0%|          | 0/50 [00:00<?, ?it/s]

Starting Training Loop...
 [1/50] [50/1582]	Loss_Critic: -140.09380859	Loss_G: -2.42577595	Mean_GP: 82.414122
 [1/50] [100/1582]	Loss_Critic: -83.34349411	Loss_G: -73.45739075	Mean_GP: 42.326853
 [1/50] [150/1582]	Loss_Critic: -44.34096092	Loss_G: -89.29139786	Mean_GP: 17.920329
 [1/50] [200/1582]	Loss_Critic: -31.15977104	Loss_G: -73.78591835	Mean_GP: 10.898851
 [1/50] [250/1582]	Loss_Critic: -23.71692211	Loss_G: -69.73328316	Mean_GP: 7.187081
 [1/50] [300/1582]	Loss_Critic: -19.04324661	Loss_G: -55.80266487	Mean_GP: 4.998261
 [1/50] [350/1582]	Loss_Critic: -16.78782141	Loss_G: -52.48507500	Mean_GP: 4.119900
 [1/50] [400/1582]	Loss_Critic: -14.79240738	Loss_G: -52.23293701	Mean_GP: 3.708163
 [1/50] [450/1582]	Loss_Critic: -13.15499739	Loss_G: -58.39624382	Mean_GP: 2.891489
 [1/50] [500/1582]	Loss_Critic: -11.70095378	Loss_G: -58.40364388	Mean_GP: 2.442743
 [1/50] [550/1582]	Loss_Critic: -10.58107088	Loss_G: -58.13599365	Mean_GP: 1.987654
 [1/50] [600/1582]	Loss_Critic: -9.69963051	Los


  2%|▏         | 1/50 [02:41<2:11:34, 161.12s/it]

 [2/50] [50/1582]	Loss_Critic: -5.95751090	Loss_G: -32.94981468	Mean_GP: 0.727487
 [2/50] [100/1582]	Loss_Critic: -5.88367435	Loss_G: -25.47918098	Mean_GP: 0.681287
 [2/50] [150/1582]	Loss_Critic: -5.84054397	Loss_G: -25.99093372	Mean_GP: 0.818854
 [2/50] [200/1582]	Loss_Critic: -6.12519587	Loss_G: -21.64142082	Mean_GP: 0.781946
 [2/50] [250/1582]	Loss_Critic: -6.21242845	Loss_G: -27.38775108	Mean_GP: 0.825591
 [2/50] [300/1582]	Loss_Critic: -5.96115464	Loss_G: -21.82356255	Mean_GP: 0.718467
 [2/50] [350/1582]	Loss_Critic: -5.83149361	Loss_G: -18.18297149	Mean_GP: 0.683546
 [2/50] [400/1582]	Loss_Critic: -6.09101240	Loss_G: -24.82251728	Mean_GP: 0.772930
 [2/50] [450/1582]	Loss_Critic: -5.93049097	Loss_G: -20.94682413	Mean_GP: 0.708322
 [2/50] [500/1582]	Loss_Critic: -5.63454433	Loss_G: -23.31543831	Mean_GP: 0.701380
 [2/50] [550/1582]	Loss_Critic: -5.71666491	Loss_G: -17.27586124	Mean_GP: 0.770531
 [2/50] [600/1582]	Loss_Critic: -5.71058618	Loss_G: -13.07077615	Mean_GP: 0.738249
 [2/5


  4%|▍         | 2/50 [05:12<2:04:16, 155.35s/it]

...

 [29/50] [50/1582]	Loss_Critic: -0.48843997	Loss_G: -79.92177802	Mean_GP: 0.188435
 [29/50] [100/1582]	Loss_Critic: -0.61579345	Loss_G: -51.25458010	Mean_GP: 0.166416
 [29/50] [150/1582]	Loss_Critic: -0.34456241	Loss_G: -60.36308483	Mean_GP: 0.182354
 [29/50] [200/1582]	Loss_Critic: -0.82830692	Loss_G: -57.95115613	Mean_GP: 0.084507
 [29/50] [250/1582]	Loss_Critic: -1.04259693	Loss_G: -85.97998386	Mean_GP: 0.216474
 [29/50] [300/1582]	Loss_Critic: -0.37720412	Loss_G: -63.25243138	Mean_GP: 0.110812
 [29/50] [350/1582]	Loss_Critic: -1.39538549	Loss_G: -101.71889692	Mean_GP: 0.249171
 [29/50] [400/1582]	Loss_Critic: -1.30985057	Loss_G: 67.37608281	Mean_GP: 0.263830
 [29/50] [450/1582]	Loss_Critic: -0.93787821	Loss_G: -49.37578881	Mean_GP: 0.160178
 [29/50] [500/1582]	Loss_Critic: -1.17066659	Loss_G: -71.02509999	Mean_GP: 0.227007
 [29/50] [550/1582]	Loss_Critic: -1.33138733	Loss_G: -126.80622434	Mean_GP: 0.200675
 [29/50] [600/1582]	Loss_Critic: -1.21947193	Loss_G: -87.41211183	Mean_GP: 0


 58%|█████▊    | 29/50 [1:13:28<53:07, 151.76s/it]

 [30/50] [50/1582]	Loss_Critic: -0.90972896	Loss_G: 41.40348488	Mean_GP: 0.156230
 [30/50] [100/1582]	Loss_Critic: -0.55399141	Loss_G: 11.58241809	Mean_GP: 0.153284
 [30/50] [150/1582]	Loss_Critic: -0.82068508	Loss_G: 37.63479529	Mean_GP: 0.141596
 [30/50] [200/1582]	Loss_Critic: -1.80883800	Loss_G: -3.68733597	Mean_GP: 0.260834
 [30/50] [250/1582]	Loss_Critic: -1.46673301	Loss_G: -16.75032444	Mean_GP: 0.266852
 [30/50] [300/1582]	Loss_Critic: -0.64522074	Loss_G: -45.28115559	Mean_GP: 0.116991
 [30/50] [350/1582]	Loss_Critic: -1.68619540	Loss_G: -5.77266488	Mean_GP: 0.197453
 [30/50] [400/1582]	Loss_Critic: -0.72250939	Loss_G: -18.16305304	Mean_GP: 0.048544
 [30/50] [450/1582]	Loss_Critic: -0.99086965	Loss_G: -31.18783545	Mean_GP: 0.238280
 [30/50] [500/1582]	Loss_Critic: -0.79180824	Loss_G: -31.20497102	Mean_GP: 0.041470
 [30/50] [550/1582]	Loss_Critic: -1.68621891	Loss_G: 1.29414803	Mean_GP: 0.237702
 [30/50] [600/1582]	Loss_Critic: -1.17308603	Loss_G: -33.50464996	Mean_GP: 0.083129



 60%|██████    | 30/50 [1:16:00<50:39, 151.99s/it]

 [31/50] [50/1582]	Loss_Critic: -0.90619975	Loss_G: -11.10881226	Mean_GP: 0.112118
 [31/50] [100/1582]	Loss_Critic: -0.80920705	Loss_G: -139.77299664	Mean_GP: 0.131930
 [31/50] [150/1582]	Loss_Critic: -1.16955797	Loss_G: -46.40738235	Mean_GP: 0.232291
